<a href="https://colab.research.google.com/github/KILjungjoon/Project_Asian_Parliament/blob/main/%EA%B5%AD%ED%9A%8C%EC%9D%98%EC%9B%90%EC%A0%95%EB%B3%B4_%EC%9C%84%ED%82%A4%ED%94%BC%EB%94%94%EC%95%84_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install tqdm

In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url="https://ko.wikipedia.org/wiki/대한민국_제21대_국회의원_선거"
# HTTP 요청을 보내고 HTML 내용을 가져오기
response = requests.get(url)
html_content = response.text

soup = BeautifulSoup(html_content, 'html.parser')

# table에는 국회의원 정보가 없음.
# # 원하는 HTML 테이블을 찾기 (예: 첫 번째 테이블)
# table = soup.find('table')

# # Pandas의 read_html 함수를 사용하여 테이블을 데이터프레임으로 읽기
# df = pd.read_html(str(table))

# # 데이터프레임 출력
# print(len(df))
soup

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-typography-survey-disabled vector-toc-available" dir="ltr" lang="ko">
<head>
<meta charset="utf-8"/>
<title>대한민국 제21대 국회의원 선거 - 위키백과, 우리 모두의 백과사전</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited

### 지역구/비례대표, 의원이름, 위키url 3가지 정보만 추출

In [10]:
from tqdm import tqdm_notebook
mp=[]
politicians = []
wikiurls = []

# 지역구 의원 정보 추출하기
for td in soup.find_all('td'):
    bgcolor = td.get('bgcolor')  # 'bgcolor' 속성 값을 가져옴
    if bgcolor in ('#004EA2', '#EF426F'):
        a_tag = td.a  # <a> 태그 가져오기
        if a_tag:
            name = a_tag.get_text()  # 이름 추출
            if name == '미래한국당' or name =='더불어시민당':     # 층위가 맞지 않는 위키피디아 코드 문제 처리
                pass
            else :
                politicians.append(name)
                wikiurls.append('https://ko.wikipedia.org' + a_tag['href'])
                mp.append("지역구")

# 비례대표 정보 추출하기
wikitree_tables = soup.find_all("table", class_="wikitable")
sixteenth_table = wikitree_tables[15]       # 16번째 table이 비례대표 테이블이다.
tr_all=sixteenth_table.find_all('tr')       # 모든 tr 태그를 추출
for tr in tr_all[1:] :                      # 비례대표 table은 1행이 컬럼명, 2~20행이 의원 명단
    td_elements = tr.find_all('td')         # 각 td 요소에서 이름과 URL 추출
    for td in td_elements:
        a_tag = td.a
        if a_tag:
            name = a_tag.get_text()         # 이름 추출
            url = "https://ko.wikipedia.org"+a_tag['href']      # URL 추출
            if name != '미래한국당' and name !='더불어시민당':
                politicians.append(name)
            wikiurls.append(url)
            mp.append("비례대표")

# 결과 출력
# for name, url in zip(politicians, wikiurls):
#     print(f'이름: {name}, URL: {url}')

# 위키urlL에서 국회의원 정보 가져오기
import re
import requests
from bs4 import BeautifulSoup

introduction=[]
for wikiurl in tqdm_notebook(wikiurls) :
    response = requests.get(wikiurl)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    if soup.find_all('p'):
        if len(soup.find_all('p')) >= 2 and soup.find_all('p')[1].text.strip():
            person = soup.find_all('p')[1].text.strip()
        else:
            person = soup.find_all('p')[0].text.strip()
    else:
        # Handle the case where there are no 'p' elements found in the soup
        person = "요소 'p' 없음"
    person=re.sub(r'\[\d+\]', '', person)
    introduction.append(person)

import pandas as pd
MPs=pd.DataFrame({'구분':mp, '이름':politicians, '소개':introduction, '위키url':wikiurls })
MPs

<ipython-input-10-9294bbdd5956>:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for wikiurl in tqdm_notebook(wikiurls) :


  0%|          | 0/294 [00:00<?, ?it/s]

,구분,이름,소개,위키url
0,지역구,이낙연,"이낙연(李洛淵, 1952년 1월 11일~)은 기자 출신 정치인이다. 제16·17·1...",https://ko.wikipedia.org/wiki/%EC%9D%B4%EB%82%...
1,지역구,홍익표,"홍익표(洪翼杓, 1967년 11월 20일~)는 대한민국의 정치인으로, 제19·20·...",https://ko.wikipedia.org/wiki/%ED%99%8D%EC%9D%...
2,지역구,박성준,"박성준(朴省俊, 1969년 4월 23일~)은 대한민국의 전직 방송인(전직 KBS대전...",https://ko.wikipedia.org/wiki/%EB%B0%95%EC%84%...
3,지역구,권영세,"권영세(權寧世, 1959년 4월 1일~)는 대한민국의 검사 출신 법조인, 정치인이다...",https://ko.wikipedia.org/wiki/%EA%B6%8C%EC%98%...
4,지역구,전혜숙,"전혜숙(全惠淑, 1955년 5월 5일~)은 대한민국 정치인이다. 제18·20·21대...",https://ko.wikipedia.org/wiki/%EC%A0%84%ED%98%...
...,...,...,...,...
289,비례대표,전용기,"전용기(田溶冀, 1991년 10월 26일~)은 대한민국의 정치인으로 더불어민주당 대...",https://ko.wikipedia.org/wiki/%EC%A0%84%EC%9A%...
290,비례대표,서정숙,"서정숙(徐正淑, 1953년 2월 18일~)은 대한민국의 약사 출신 정치인으로, 제2...",https://ko.wikipedia.org/wiki/%EC%84%9C%EC%A0%...
291,비례대표,양경숙,"양경숙(梁敬淑, 1962년 9월 6일~)은 대한민국의 시의원 출신 정치인이다. 전북...",https://ko.wikipedia.org/wiki/%EC%96%91%EA%B2%...
292,비례대표,이용,"이용(李鏞, 1978년 6월 23일~)은 대한민국의 은퇴한 썰매(루지, 스켈레톤, ...",https://ko.wikipedia.org/wiki/%EC%9D%B4%EC%9A%...


In [11]:
MPs.to_excel("/content/drive/MyDrive/Python Code/21대국회의원정보_위키피디아.xlsx", index=False)

## 소스 코드
* 위의 main code가 정확한 최종 코드.

In [22]:
# 지역구 의원 명단, 위키피디아url 추출하기
politicians = []
wikiurl = []

for td in soup.find_all('td', {'bgcolor': '#004EA2'}):
    link = td.find('a')
    if link is not None:  # link가 None이 아닌 경우에만 처리
        politicians.append(link.text)
        wikiurl.append('https://ko.wikipedia.org' + link.get('href'))

print(politicians)
print(len(politicians))
print(wikiurl)

['이낙연', '홍익표', '박성준', '전혜숙', '고민정', '안규백', '장경태', '서영교', '박홍근', '김영배', '기동민', '천준호', '박용진', '인재근', '오기형', '고용진', '우원식', '김성환', '박주민', '강병원', '우상호', '김영호', '노웅래', '정청래', '황희', '이용선', '강선우', '진성준', '한정애', '이인영', '윤건영', '최기상', '김영주', '김민석', '김병기', '이수진', '유기홍', '정태호', '남인순', '진선미', '이해식', '박재호', '전재수', '최인호', '허종식', '박찬대', '정일영', '맹성규', '윤관석', '이성만', '홍영표', '유동수', '송영길', '김교흥', '신동근', '윤영덕', '이병훈', '송갑석', '양향자', '조오섭', '이형석', '이용빈', '민형배', '장철민', '황운하', '박병석', '박범계', '조승래', '이상민', '박영순', '이상헌', '홍성국', '강준현', '김승원', '백혜련', '김영진', '박광온', '김진표', '김태년', '윤영찬', '김병욱', '오영환', '김민철', '강득구', '민병덕', '이재정', '김경협', '설훈', '김상희', '서영석', '임오경', '양기대', '홍기원', '전해철', '김철민', '고영인', '김남국', '한준호', '홍정민', '이용우', '이소영', '윤호중', '조응천', '김한정', '김용민', '안민석', '문정복', '조정식', '이학영', '최종윤', '김민기', '정춘숙', '이탄희', '윤후덕', '박정', '이규민', '김주영', '박상혁', '송옥주', '이원욱', '권칠승', '소병훈', '임종성', '정성호', '허영', '이광재', '송기헌', '정정순', '이장섭', '도종환', '변재일', '임호선', '문진석', '박완주', '이정문', '강훈식', '김종민', '어기구', '김윤덕', '이상직', '김성주', '신영대', '김수흥', '한

In [ ]:
# 비례대표 의원 명단, 위키피디아url 추출하기
from bs4 import BeautifulSoup

# class가 "wikitree"인 모든 table 요소를 찾습니다.
wikitree_tables = soup.find_all("table", class_="wikitable")

# 16번째 table 요소를 출력합니다.
if len(wikitree_tables) >= 16:
    sixteenth_table = wikitree_tables[15]  # Python 인덱스는 0부터 시작하므로 16번째 요소는 15입니다.
    print(sixteenth_table)
else:
    print("16번째 table을 찾을 수 없습니다. 총 table 개수:", len(wikitree_tables))

In [ ]:
# 비례대표 table은 1행이 컬럼명, 2~20행이 의원 명단
# 모든 tr 태그를 추출
tr_all=sixteenth_table.find_all('tr')

politicians = []
wikiurls = []
for tr in tr_all[1:] :

    # 각 td 요소에서 이름과 URL 추출
    td_elements = tr.find_all('td')
    for td in td_elements:
        a_tag = td.a
        if a_tag:
            name = a_tag.get_text()  # 이름 추출
            url = "https://ko.wikipedia.org"+a_tag['href']      # URL 추출
            politicians.append(name)
            wikiurls.append(url)

# 결과 출력
for name, url in zip(politicians, wikiurls):
    print(f'이름: {name}, URL: {url}')

In [4]:
# 위키피디아에서 인물 소개 첫 문단 가져오기
import re
import requests
from bs4 import BeautifulSoup

url = "https://ko.wikipedia.org/wiki/%ED%99%8D%EC%9D%B5%ED%91%9C_(1967%EB%85%84)"

# URL에서 HTML 가져오기
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

# 국민의당 '김희곤' 의원의 경우만 p태그가 1개 있음
# 따라서 else로 처리해야 함.
if soup.find_all('p'):
    if len(soup.find_all('p')) >= 2 and soup.find_all('p')[1].text.strip():
        person = soup.find_all('p')[1].text.strip()
    else:
        person = soup.find_all('p')[0].text.strip()
else:
    # Handle the case where there are no 'p' elements found in the soup
    person = "요소 'p' 없음"
person=re.sub(r'\[\d+\]', '', person)
person

'홍익표(洪翼杓, 1967년 11월 20일~)는 대한민국의 정치인으로, 제19·20·21대 국회의원이다. 2023년 9월 26일, 더불어민주당 원내대표에 선출되었다. 더불어민주당 수석 대변인과 제19대 문재인 대통령 후보 수석대변인, 국정기획자문위원회 기획분과위원, 더불어민주당 정책위원회 수석부의장 등을 역임하였다.'